## Data Modeling
Describe and justify the process for analyzing or modeling the data.

Questions to consider:

How did you analyze or model the data?
How did you iterate on your initial approach to make it better?
Why are these choices appropriate given the data and the business problem?

# Contents
1. Load the dataset
2. Baseline model
3. Model Iterations

## 1.Loading Dataset

In [1]:
from PIL import Image
import os
import numpy as np

def get_image_sizes(directory):
    """Collect widths and heights of images with given extensions in directory."""
    sizes = [(img.size) for root, dirs, files in os.walk(directory)
             for file in files if file.lower().endswith('jpeg')
             for img in (Image.open(os.path.join(root, file)),)]
    return zip(*sizes)  # Unzips the sizes into two lists: widths and heights

def print_image_statistics(widths, heights):
    """Print statistics for a collection of image widths and heights."""
    num_images = len(widths)
    avg_width = round(np.mean(widths))
    avg_height = round(np.mean(heights))
    avg_aspect_ratio = np.mean(widths) / np.mean(heights)
    max_width = max(widths)
    max_height = max(heights)
    min_width = min(widths)
    min_height = min(heights)
    
    print(f"Number of images: {num_images}")
    print(f"Average width: {avg_width}")
    print(f"Average height: {avg_height}")
    print(f"Average aspect ratio: {avg_aspect_ratio:.2f}")
    print(f"Max width: {max_width}")
    print(f"Max height: {max_height}")
    print(f"Min width: {min_width}")
    print(f"Min height: {min_height}")
    return avg_aspect_ratio

train_dir = 'data/chest_xray/new_train/'
test_dir = 'data/chest_xray/new_test/'

train_widths, train_heights = get_image_sizes(train_dir)
test_widths, test_heights = get_image_sizes(test_dir)

widths = np.concatenate((train_widths, test_widths))
heights = np.concatenate((train_heights, test_heights))

avg_ratio = print_image_statistics(widths, heights)

Number of images: 5271
Average width: 1330
Average height: 972
Average aspect ratio: 1.37
Max width: 2890
Max height: 2713
Min width: 384
Min height: 127


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
tf.random.set_seed(42)
import warnings
warnings.filterwarnings('ignore')

def create_dataset(directory, width, batch_size, ratio = avg_ratio):
    height = int(width/ratio)
    dataset = image_dataset_from_directory(directory,
                                        label_mode='binary',
                                        color_mode="grayscale", # will save memory as images are already in grayscale
                                        batch_size=batch_size,
                                        image_size=(height,width),
                                        shuffle=True,
                                        crop_to_aspect_ratio = True,
                                        seed=42)
    return dataset
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def process_dataset(dataset):
    # Define the rescaling layer
    rescale = Rescaling(1./255)
    
    # Normalizing the dataset
    dataset = dataset.map(lambda x, y: (rescale(x), y))
    
    # improves speed by only having to read the dataset for the first epoch
    dataset = dataset.cache()
    
    # increases generalization by shuffling elements each epoch
    dataset = dataset.shuffle(buffer_size=1000, seed=42)
    
    # this automatically adjusts the number of batches
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

2024-02-29 11:38:06.372205: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
train_dir = "data/chest_xray/new_train"
val_dir = "data/chest_xray/new_val"
test_dir = "data/chest_xray/new_test"


train_ds = create_dataset(train_dir, width=256, batch_size=32,ratio = avg_ratio)
val_ds = create_dataset(val_dir, width=256, batch_size=32,ratio = avg_ratio)
test_ds = create_dataset(test_dir, width=256, batch_size=32,ratio = avg_ratio)

train_ds_scaled = process_dataset(train_ds)
val_ds_scaled = process_dataset(val_ds)
test_ds_scaled = process_dataset(test_ds)

input_shape = (187, 256, 1)

Found 4684 files belonging to 2 classes.
Found 585 files belonging to 2 classes.
Found 587 files belonging to 2 classes.


In [9]:
train_ds_scaled

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 187, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

For metrics, I chose accuracy to show overall performance of the model, Recall to emphasise the importance of minimizing false negatives, and AUC. 

In [11]:
from tensorflow.keras.metrics import BinaryAccuracy, Recall, AUC
metrics=[BinaryAccuracy(name='accuracy'),
         Recall(name='recall'),
         AUC(name='auc')]

## Baseline Model

In [12]:
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
import time

baseline_model = Sequential([
    Flatten(input_shape=input_shape),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])
baseline_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
start_time = time.time()
baseline_results = baseline_model.fit(train_ds_scaled,
                                      epochs=10,
                                      validation_data=val_ds_scaled,
                                      verbose=0)
end_time = time.time()
duration = end_time - start_time
print(f"Training time: {duration} seconds")
print('')

Training time: 19.46078085899353 seconds



In [15]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.optimizers import SGD
import time
import random
random.seed(42)
import numpy as np
np.random.seed(42)
tf.random.set_seed(42)

metrics=[tf.keras.metrics.AUC(curve = 'PR', name='auc_pr'),
         tf.keras.metrics.AUC(name='auc_'),
         'accuracy',
         tf.keras.metrics.Precision(name='precision'),
         tf.keras.metrics.Recall(name='recall')]

def visualize_training_results(history,num_epochs):
    metric = list(history.history.keys())[1]
    train_score = history.history[metric]
    val_score = history.history['val_'+metric]
    
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(num_epochs)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_score, label='Training '+metric)
    plt.plot(epochs_range, val_score, label='Validation '+metric)
    plt.legend(loc='lower right')
    plt.title('Training and Validation '+metric)
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, train_loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
def baseline_modeler(model, metrics = 'accuracy', optimizer = 'adam', num_epochs=100, early_stopping = None ):
    
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=[metrics])
    start_time = time.time()
    if early_stopping == None:
        history = model.fit(train_ds,
                            epochs=num_epochs,
                            validation_data=val_ds,
                            verbose=0)
    else:
        history = model.fit(train_ds,
                            epochs=num_epochs,
                            validation_data=val_ds,
                            verbose=0,
                            callbacks = [early_stopping])
        
    end_time = time.time()
    duration = end_time - start_time
    print(f"Training time: {duration} seconds")
    print('')
    visualize_training_results(history, num_epochs=int(len(history.history['loss'])))
    print('')
    train_scores = model.evaluate(train_ds)
    val_scores = model.evaluate(val_ds)
    num_metrics = int(len(history.history.keys())/2)
    metrics_names = list(history.history.keys())[:num_metrics]
    diff_scores = [b - a for a, b in zip(train_scores, val_scores)]
    display(pd.DataFrame([train_scores,val_scores,diff_scores],index=['Train','Val','Diff'],columns=metrics_names))
    print('------------------------------')
    print('')

def create_dataset(directory):
    dataset = image_dataset_from_directory(directory,
                                        label_mode='binary',
                                        color_mode="grayscale", # will save memory as images are already in grayscale
                                        batch_size=32, # selecting default value
                                        image_size=(256,256), # selecting default value
                                        shuffle=True,
                                        seed=42)
    return dataset
def process_dataset(dataset):
    # Define the rescaling layer
    rescale = Rescaling(1./255)
    
    # Normalizing the dataset
    dataset = dataset.map(lambda x, y: (rescale(x), y))
    
    # improves speed by only having to read the dataset for the first epoch
    dataset = dataset.cache()
    
    # increases generalization by shuffling elements each epoch
    dataset = dataset.shuffle(buffer_size=1000, seed=42)
    
    # this automatically adjusts the number of batches
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [16]:
# Paths to the images
train_dir = "data/chest_xray/new_train"
test_dir = "data/chest_xray/new_test"
val_dir = "data/chest_xray/new_val"

train_ds_raw = create_dataset(train_dir)
test_ds_raw = create_dataset(test_dir)
val_ds_raw = create_dataset(val_dir)

train_ds = process_dataset(train_ds_raw)
test_ds = process_dataset(test_ds_raw)
val_ds = process_dataset(val_ds_raw)

Found 4684 files belonging to 2 classes.
Found 587 files belonging to 2 classes.
Found 585 files belonging to 2 classes.


In [ ]:
model = Sequential([
    Rescaling(1./255, input_shape= input_shape),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=metrics)
results = model.fit(train_ds,
                    epochs=10,
                    validation_data=val_ds)

In [7]:
# Baseline, no initializers
model = Sequential([
    Rescaling(1./255, input_shape= input_shape),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
# model.summary()
# results = model_1.fit(train_ds, 
#                         train_labels,
#                         epochs=50,
#                         validation_data=val_ds)
from tensorflow.keras.models import load_model

# Path to the saved model
model_path = 'models/1_baseline.h5'

# Load the model
model = load_model(model_path)

# After loading, you can use the model to make predictions, evaluate it, etc.
train_scores = model.evaluate(train_ds)
val_scores = model.evaluate(val_ds)


10/10 [==============================] - 0s 39ms/step - loss: 0.2287 - auc_pr: 0.9882 - auc_: 0.9800 - accuracy: 0.9607 - precision: 0.9676 - recall: 0.9789


In [9]:
train_scores = model.evaluate(train_ds)
val_scores = model.evaluate(val_ds)


10/10 [==============================] - 1s 50ms/step - loss: 0.2287 - auc_pr: 0.9882 - auc_: 0.9800 - accuracy: 0.9607 - precision: 0.9676 - recall: 0.9789


In [11]:


display(pd.DataFrame([val_scores],columns=['loss', 'auc_pr','auc','accuracy','precicion','recall']))

,loss,auc_pr,auc,accuracy,precicion,recall
0,0.22866,0.988204,0.980034,0.960684,0.967593,0.978923


In [ ]:
def buildModel(optimizer):
    # Initialising the ANN
    classifier = models.Sequential()
    
    # Adding the input layer and the first hidden layer
    classifier.add(layers.Dense(128, activation = 'relu', input_shape=(49152,))) 
    
    # Adding the second hidden layer
    classifier.add(layers.Dense(32, activation='relu'))
        
    # Adding the output layer
    classifier.add(layers.Dense(5, activation='sigmoid'))
    
    # Compiling the ANN
    classifier.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
    
    return classifier

In [ ]:
binary_classifier = KerasRegressor(build_fn = buildModel_binary)
#What hyperparameter we want to play with
parameters = {'batch_size': [16, 32, 64, 128],
              'epochs': [5, 10, 50, 100],
              'optimizer': ['adam', 'sgd' 'rmsprop']}
binary_grid_search = GridSearchCV(estimator = binary_classifier,
                           param_grid = parameters,
                           scoring = 'neg_mean_absolute_error',
                           cv = 5)
binary_grid_search = binary_grid_search.fit(binary_train, binary_train_labels, verbose = 0

In [ ]:
# Same model with he_normal initializer
# Best initializer based on val_loss value, despite overfitting for all
model = Sequential([
    
    Rescaling(1./255, input_shape= input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'adam', num_epochs=50, early_stopping = None )

In [ ]:
# lecun_normal initializer

model = Sequential([
    
    Rescaling(1./255, input_shape= input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='lecun_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='lecun_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='lecun_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='lecun_normal'),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'adam', num_epochs=50, early_stopping = None )

In [ ]:
# SGD optimizer with he_normal initializer
# Beat out adam by a good amount and killed rmsprop

model = Sequential([
    
    Rescaling(1./255, input_shape= input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=50, early_stopping = None )

In [ ]:
# RMSprop optimizer with he_normal initializer
model = Sequential([
    
    Rescaling(1./255, input_shape= input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'rmsprop', num_epochs=50, early_stopping = None )

In [ ]:
# l1 regularizer, SGD optimizer, he_normal initializer

model = Sequential([
    
    Rescaling(1./255, input_shape= input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',kernel_regularizer=l1(0.01)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',kernel_regularizer=l1(0.01)),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',kernel_regularizer=l1(0.01)),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal',kernel_regularizer=l1(0.01)),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=50, early_stopping = None )

In [ ]:
# l2 regularizer, SGD optimizer, he_normal initializer

model = Sequential([
    
    Rescaling(1./255, input_shape= input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal',kernel_regularizer=l2(0.01)),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=50, early_stopping = None )

In [ ]:
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    Dropout(0.2),  # Example dropout after the first conv layer
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    Dropout(0.3),  # Slightly higher dropout after the second conv layer
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    Dropout(0.4),  # Even higher dropout after the third conv layer
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=300, early_stopping = early_stopping)

In [ ]:
# Dropout after dense
# SGD, he_normal
early_stopping = EarlyStopping(monitor='val_loss', 
                               min_delta=0.001, 
                               patience=10, 
                               mode='min', 
                               restore_best_weights=True,
                               verbose=1)
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),  
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=300, early_stopping = early_stopping)

In [ ]:
# Dropout after pooling
# SGD, he_normal
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),  # Adding dropout after pooling
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),  # Repeating pattern for consistency
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),  # Maintaining dropout after pooling
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=300, early_stopping = early_stopping)

In [ ]:
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=300, early_stopping = early_stopping )

In [ ]:
# Increasing Filter Size in Conv2D Layers
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    Dropout(0.1),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=100, early_stopping = early_stopping)

In [ ]:
# Varying Kernel Size
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (5, 5), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (5, 5), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (5, 5), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),  # Adding dropout before the final dense layer
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=200, early_stopping = early_stopping )

In [ ]:
# Adjusting Units in Dense Layers
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),

    Dense(256, activation='relu', kernel_initializer='he_normal'),#
    Dropout(0.5), 
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=200, early_stopping = early_stopping )

In [ ]:
# Experimenting with Dropout Rates
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),

    Dense(128, activation='relu', kernel_initializer='he_normal'),#
    Dropout(0.3), 
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=200, early_stopping = early_stopping )

In [ ]:
# Adding an additional Conv2D + MaxPooling2D block
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),  # Adding dropout before the final dense layer
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=200, early_stopping = early_stopping )

In [ ]:
# Adding an additional Conv2D + MaxPooling2D block
model = Sequential([
    Rescaling(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),  # Adding dropout after new Conv2D block
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),  # Adding dropout before the final dense layer
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'SGD', num_epochs=200, early_stopping = early_stopping )

In [ ]:
# Adding an additional Dense layer
model = Sequential([
    RandomZoom(1./255, input_shape=input_shape),
    
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])
baseline_modeler(model, metrics = metrics, optimizer = 'adam', num_epochs=200, early_stopping = early_stopping )

In [18]:
from tensorflow.keras.models import load_model

# Path to the saved model
model_path = 'models/26_best_model.h5'

# Load the model
model = load_model(model_path)

model

In [ ]:
# After loading, you can use the model to make predictions, evaluate it, etc.
train_scores = model.evaluate(train_ds_scaled)
val_scores = model.evaluate(val_ds_scaled)